In [6]:
import requests
import json
import prettytable
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['LEU0252917300','LEU0254929400','LEU0252918500','LEU0252919100','LEU0252919700']
                   ,"startyear":"2014", "endyear":"2019"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
            if 'M01' <= period <= 'M12':
                x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()   

In [9]:
import pandas as pd
import json
import requests

def get_bls_data(series, start, end):
    headers = {'Content-Type': 'application/json'}
    data = json.dumps({"LEU0252919700": series,"2014":"%d" % (start), "2019":"%d" % (end)})
    p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text)
    try:
        df = pd.DataFrame()
        for series in json_data['Results']['series']:
            df_initial = pd.DataFrame(series)
            series_col = df_initial['seriesID'][0]
            for i in range(0, len(df_initial) - 1):
                df_row = pd.DataFrame(df_initial['data'][i])
                df_row['seriesID'] = series_col
                if 'code' not in str(df_row['footnotes']): 
                    df_row['footnotes'] = ''
                else:
                    df_row['footnotes'] = str(df_row['footnotes']).split("'code': '",1)[1][:1]
                df = df.append(df_row, ignore_index=True)
        return df
    except:
        json_data['status'] == 'REQUEST_NOT_PROCESSED'
        print('BLS API has given the following Response:', json_data['status'])
        print('Reason:', json_data['message'])


start = 2014
end = 2019
#series = [LEU0252881800','LEU0252882700','LEU0252916700','LEU0252917300','LEU0254929400',
 #        'LEU0252918500','LEU0252919100', 'LEU0252919700','LAUST040000000000005'
#]
series = ['LEU0252919700']

df = get_bls_data(series=series, start=start, end=end)
writer = pd.ExcelWriter('bls.xlsx', engine='xlsxwriter', options={'strings_to_numbers': True})
df.to_excel(writer, sheet_name='Sheet1', index=False)
#writer.save()

BLS API has given the following Response: REQUEST_FAILED
Reason: ['Your request has failed. Please check your input parameters, and try your request again.']


AttributeError: 'NoneType' object has no attribute 'to_excel'

# Performing a GET Request and saving the 
# API's response within a variable

In [5]:
import pandas as pd
import requests
import json
import prettytable
url = 'https://api.bls.gov/publicAPI/v1/timeseries/data/LEU0252917300', 
# using the Inflation & Prices average price series
data = requests.get(url).json()
response = requests.get(url)
response_json = response.json()
#print('Status: ' + data['status'])
#print(json.dumps(response_json, indent=4, sort_keys=True))

In [2]:
#print(data.keys())
